In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
russ_train_location = "./data/Russ_994_random.fasta"
tau_train_location = "./data/tautomerase_2953.fasta"
output_dir = "./outputs/graphs/"

In [ ]:
# cd ../../

In [ ]:
import pgen.utils as utils
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Align import substitution_matrices
 
sub_matrices = {
    "BLOSUM62": substitution_matrices.load("BLOSUM62"),
    "BLOSUM90": substitution_matrices.load("BLOSUM90")
}

def align_sequences(sequenceA, sequenceB, print_alignment=False, matrix="BLOSUM90"):
  if matrix is not None: 
    match_dict = sub_matrices[matrix]
    alignments = pairwise2.align.globaldx(sequenceA=sequenceA, sequenceB=sequenceB, match_dict=match_dict)
  else:
    alignments = pairwise2.align.globalxx(sequenceA=sequenceA, sequenceB=sequenceB)
  
  if print_alignment:
    print(format_alignment(*alignments[0]))

  return (alignments[0].score)

In [ ]:
russ_values = {
    "Russ": "outputs/models/russ/russ_gen.fasta",
    "CM2_Tst": "data/Russ_226_random.fasta",
    "Tau_Tst": "data/tautomerase_738.fasta",
    "1_S": "outputs/models/ESM34/esm34_CH_2_e_coli_full_sequential_k0.fasta",
    "1_R": "outputs/models/ESM34/E_coli_parallel_esm34.fasta",
    "1_B": "outputs/models/ESM34/E_coli_parallel_burnin_esm34.fasta",
    "1_C": "outputs/models/ESM34/E_coli_CM2_first_20_2549bd_trial1.fasta",
    "1_N": "outputs/models/ESM34/NoSeed_2549bd_trial2.fasta",
    "2_B": "outputs/models/esm1_t12_85M_UR50S/E_coli_parallel_burnin_esm12.fasta",
    "3_B": "outputs/models/esm1_t12_85M_UR50S/finetuned/Russ_994_random/Dec04_13-47-23_6fb55ea4/generated_sequences/E_coli_CM2_6b9c30_trial5.fasta",
    "3_C": "outputs/models/esm1_t12_85M_UR50S/finetuned/Russ_994_random/Dec04_13-47-23_6fb55ea4/generated_sequences/E_coli_CM2_first_20_7d2781_trial1.fasta",
    "3_N": "outputs/models/esm1_t12_85M_UR50S/finetuned/Russ_994_random/Dec04_13-47-23_6fb55ea4/generated_sequences/NoSeed_83abbe_trial2.fasta",
    "4_B": "outputs/models/esm1_t12_85M_UR50S/finetuned/tautomerase_2953/Dec04_14-13-14_c8588459/generated_sequences/E_coli_CM2_bb1f59_trial5.fasta",
    "5_B": "Outputs/models/ProtBert-BFD/E_coli_CM2_db515f_trial1.fasta",
    "6_N": "outputs/models/3_gram_CM_2/Russ_994_random_3gram_generation_seed_s_len92.fasta",
    "HMM": "outputs/models/hmmemit/CM_2_hmmemit.fasta"
}

tau_values = {
    "Tau_Tst": "data/tautomerase_738.fasta",
    "4_B": "outputs/models/esm1_t12_85M_UR50S/finetuned/tautomerase_2953/Dec04_14-13-14_c8588459/generated_sequences/E_coli_CM2_bb1f59_trial5.fasta",
    "4_N": "outputs/models/esm1_t12_85M_UR50S/finetuned/tautomerase_2953/Dec04_14-13-14_c8588459/generated_sequences/NoSeed_30995f_trial2.fasta",
    "7_N": "outputs/models/3_gram_Tautomerase/tautomerase_2953_3gram_generation_seed_s_len60.fasta"
}



# Compare to training data

In [ ]:
def process_key(file_loc, train_sequence):
    try:
        samples = utils.parse_fasta(file_loc)
    except Exception as e:
        print(f"skipping file:{file_loc}")
        return [],[]
    results = []
    print(key)
    for sample in samples[:50]:
        best = 0
        for ref_sequence in train_sequence:
            score = align_sequences(ref_sequence, sample, matrix=None)
            best = max(best, score)
            results.append(best)
    return samples, results

In [ ]:
# russ_samples = {}
# russ_results = {}
# tau_samples = {}
# tau_results = {}

# russ_train = utils.parse_fasta(russ_train_location)
# tau_train = utils.parse_fasta(tau_train_location)

# for key in russ_values:
#     sample,result = process_key(russ_values[key], russ_train)
#     russ_samples[key] = sample
#     russ_results[key] = result
    
# for key in tau_values:
#     sample,result = process_key(tau_values[key], tau_train)
#     tau_samples[key] = sample
#     tau_results[key] = result
    

In [ ]:
russ_results.keys()

In [ ]:
tau_results.keys()

In [ ]:
# Backup the data
!pip install dill
import dill
with open(output_dir + "alignment_backup", 'wb') as backup_file:
  dill.dump((russ_samples, russ_results, tau_samples, tau_results), backup_file)

# Load the data
# import dill
# with open(output_dir + "alignment_backup", 'rb') as backup_file:
#   data = dill.load(backup_file)

In [ ]:
plt.clf()
df = pd.DataFrame()
scores = list()
datasets = list()
for k,v in russ_results.items():
    scores += v
    datasets += [k] * len(v)
df["dataset"] = datasets
df["Identity Score"] = scores

sns.set(rc={'figure.figsize':(20,4)})
ax = sns.boxplot(x="dataset", y="Identity Score", data=df)
ax.figure.set_size_inches(20,5)
ax.figure.savefig(output_dir + "identity_wrt_most_aligned_training_value_russ.png")

In [ ]:
plt.clf()
df = pd.DataFrame()
scores = list()
datasets = list()
for k,v in russ_results.items():
    scores += v
    datasets += [k] * len(v)
for k,v in tau_results.items():
    scores += v
    datasets += [k + "*"] * len(v)
df["dataset"] = datasets
df["Identity Score"] = scores

sns.set(rc={'figure.figsize':(20,4)})
ax = sns.boxplot(x="dataset", y="Identity Score", data=df)
ax.figure.set_size_inches(20,5)
ax.figure.savefig(output_dir + "identity_wrt_most_aligned_training_value_both.png")

In [ ]:
plt.clf()
df = pd.DataFrame()
scores = list()
datasets = list()
for k,v in tau_results.items():
    scores += v
    datasets += [k] * len(v)
df["dataset"] = datasets
df["Identity Score"] = scores

sns.set(rc={'figure.figsize':(6,6)})
ax = sns.boxplot(x="dataset", y="Identity Score", data=df)
ax.figure.set_size_inches(6,5)
ax.figure.savefig(output_dir + "identity_wrt_most_aligned_training_value_tautomerase.png")